# 06-state-machine.ipynb

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

In [ ]:
# Agent 내부의 Tool 활용에서 공통적으로 사용할 양식(State) 생성

from langchain.agents import AgentState
# typing, pydantic, dataclass -> 라는 말들은 데이터의 형태를 설명하기 위한 애들
from typing_extensions import NotRequired
from typing import Literal


# SupportStep 은 말그대로, [a, b, c] 중에 하나임
SupportStep = Literal['warranty_collector', 'issue_checker', 'solution_maker']


class SupportState(AgentState):
    """고객 지원 업무 흐름을 위한 State"""    
    # 현재 단계
    current_step: NotRequired[SupportStep]  # 비어있어도 됨 -> 내용이 있다면 SupportStep -> 위 3가지 중 하나
    # 보증 상태
    warranty_status: NotRequired[Literal['in_warranty', 'out_of_warranty']]  # LLM 은 자연어를 제일 좋아함
    # 이슈 타입
    issue_type: NotRequired[Literal['hw', 'sw']]


In [ ]:
# Tool 만들기 (중급)
# 기존: Agent -> Tool 답변, 정보, 데이터 | 현재: Tool -> Agent에게 명령

from langchain.tools import tool, ToolRuntime
from langchain.messages import ToolMessage
from langgraph.types import Command  # langchain agent 가 langgraph 기반!


@tool
def record_warranty_status(
    status: Literal['in_warranty', 'out_of_warranty'],
    runtime: ToolRuntime[None, SupportState]
) -> Command:
    """사용자 보증상태를 기록하고, issue_checker 로 넘김"""
    
    update_content = {
        'messages': [
            ToolMessage(
                content=f'보증 상태가 확인되었습니다: {status}',
                tool_call_id=runtime.tool_call_id
            )
        ],
        'warranty_status': status,
        'current_step': 'issue_checker'  # 다음 Tool은 무엇인지 명시 (next?)
    }
    return Command(update=update_content)  # Agent에게 시킬 일을 return


@tool
def record_issue_type(
    issue_type: Literal["hw", "sw"],
    runtime: ToolRuntime[None, SupportState],
) -> Command:  
    """이슈 타입을 기록하고, solution_maker에게 전달."""
    return Command(  
        update={  
            "messages": [
                ToolMessage(
                    content=f"이슈 종류는: {issue_type}",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
            "issue_type": issue_type,
            "current_step": "solution_maker",  
        }
    )


@tool
def escalate_to_human(reason: str) -> str:
    """Escalate the case to a human support specialist."""
    # 실제 전문가에게 노티 보내기
    return f"전문가 지원으로 넘김. 사유: {reason}"


@tool
def provide_solution(solution: str) -> str:
    """Provide a solution to the customer's issue."""
    # 실제로는 해결 방법을 검색해서 보내줌
    return f"해결 방안을 전달: {solution}"

In [ ]:
# 각 Step 마다 필요한 설정 하기

# 이전: 전체 작업 SystemPrompt | 현재: 각 Step마다 다른 프롬프트
WARRANTY_COLLECTOR_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Warranty verification

At this step, you need to:
1. Greet the customer warmly
2. Ask if their device is under warranty
3. Use record_warranty_status to record their response and move to the next step

Be conversational and friendly. Don't ask multiple questions at once."""


ISSUE_CHECKER_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Issue classification
CUSTOMER INFO: Warranty status is {warranty_status}

At this step, you need to:
1. Ask the customer to describe their issue
2. Determine if it's a hardware issue (physical damage, broken parts) or software issue (app crashes, performance)
3. Use record_issue_type to record the classification and move to the next step

If unclear, ask clarifying questions before classifying."""


SOLUTION_MAKER_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Resolution
CUSTOMER INFO: Warranty status is {warranty_status}, issue type is {issue_type}

At this step, you need to:
1. For SW issues: provide troubleshooting steps using provide_solution
2. For HW issues:
   - If IN WARRANTY: explain warranty repair process using provide_solution
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

Be specific and helpful in your solutions."""


# Step configuration: maps step name to (prompt, tools, required_state)
STEP_CONFIG = {
    "warranty_collector": {
        "prompt": WARRANTY_COLLECTOR_PROMPT,
        "tools": [record_warranty_status],
        "requires": [],
    },
    "issue_checker": {
        "prompt": ISSUE_CHECKER_PROMPT,
        "tools": [record_issue_type],
        "requires": ["warranty_status"],
    },
    "solution_maker": {
        "prompt": SOLUTION_MAKER_PROMPT,
        "tools": [provide_solution, escalate_to_human],
        "requires": ["warranty_status", "issue_type"],
    },
}

In [ ]:
# Step 기반의 미들웨어 만들기
# Middleware -> 작업 순서에서 특정 시점에 반드시 실행해야 하는 것
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable


@wrap_model_call
def apply_step_config(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    """Configure agent behavior based on the current step."""
    # 현재 step 확인하기(시작-기본값은 warranty_collector 로 설정)
    current_step = request.state.get("current_step", "warranty_collector")  

    # 현재 step 에서 해야할 일을 확인
    stage_config = STEP_CONFIG[current_step]  

    # 반드시 필요한 정보(보증상태, 이슈분류(HW, SW)) 검증
    for key in stage_config["requires"]:
        if request.state.get(key) is None:
            raise ValueError(f"{key} must be set before reaching {current_step}")

    # 시스템 프롬프트를 생성 (프롬프트 에서 {변수명} 부분을 지금 채우는 과정)
    system_prompt = stage_config["prompt"].format(**request.state)

    # 각 스텝마다, 시스템 프롬프트를 현재 상황(사용할 Tool)에 맞게 덮어 쓴다.
    request = request.override(  
        system_prompt=system_prompt,  
        tools=stage_config["tools"],  
    )

    return handler(request)

In [ ]:
# 에이전트 만들기
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# Collect all tools from all step configurations
all_tools = [
    record_warranty_status,
    record_issue_type,
    provide_solution,
    escalate_to_human,
]

agent = create_agent(
    model,
    tools=all_tools,
    state_schema=SupportState,  
    middleware=[apply_step_config],  
    checkpointer=InMemorySaver(),  
)

In [ ]:
# Test (스레드 id 바꾸고 싶을때만 실행)

import uuid

thread_id = uuid.uuid4()
config = {'configurable': {'thread_id': thread_id}}

In [ ]:
from langchain.messages import HumanMessage

print('--- 시작 ---')
result = agent.invoke(
    {'messages': [
        {'role': 'user', 'content': '핸드폰 이상해..'},
    ]},
    config
)

for msg in result['messages']:
    msg.pretty_print()

--- 시작 ---
================================ Human Message =================================

핸드폰 이상해..
================================== Ai Message ==================================

안녕하세요! 도움을 드리게 되어 반갑습니다. 사용 중이신 핸드폰이 아직 보증 기간 내에 있으신가요?
Current step: None


In [20]:
print('--- 보증 사실관계 확인 ---')
result = agent.invoke(
    {'messages': [
        {'role': 'user', 'content': '3년 됐어..'},
    ]},
    config
)

for msg in result['messages']:
    msg.pretty_print()

print(f"Current step: {result.get('current_step')}")

--- 보증 사실관계 확인 ---
================================ Human Message =================================

핸드폰 이상해..
================================== Ai Message ==================================

안녕하세요! 도움을 드리게 되어 반갑습니다. 사용 중이신 핸드폰이 아직 보증 기간 내에 있으신가요?
================================ Human Message =================================

그게 뭐야..?
================================== Ai Message ==================================

보증 기간이라는 것은 제품을 구매하신 후 일정 기간 동안 무상으로 수리나 교환을 받을 수 있는 기간을 말해요. 사용하시는 핸드폰이 아직 그 기간 안에 있는지 궁금해서요. 혹시 구매하신 지 얼마나 되셨나요?
================================ Human Message =================================

3년 됐어..
================================== Ai Message ==================================
Tool Calls:
  record_warranty_status (call_KsNimA7FlFDFsKwfDiC74OC2)
 Call ID: call_KsNimA7FlFDFsKwfDiC74OC2
  Args:
    status: out_of_warranty
================================= Tool Message =================================
Name: record_warranty_status

보증 상태가 확인되었습니다: out_of_warranty
====

In [22]:
print('--- 문제상황 확인 ---')
result = agent.invoke(
    {'messages': [
        HumanMessage('몰라 그냥 느리고 볼륨 조절도 안됨..'),
    ]},
    config
)

for msg in result['messages']:
    msg.pretty_print()

print(f"Current step: {result.get('current_step')}")

--- 문제상황 확인 ---
================================ Human Message =================================

핸드폰 이상해..
================================== Ai Message ==================================

안녕하세요! 도움을 드리게 되어 반갑습니다. 사용 중이신 핸드폰이 아직 보증 기간 내에 있으신가요?
================================ Human Message =================================

그게 뭐야..?
================================== Ai Message ==================================

보증 기간이라는 것은 제품을 구매하신 후 일정 기간 동안 무상으로 수리나 교환을 받을 수 있는 기간을 말해요. 사용하시는 핸드폰이 아직 그 기간 안에 있는지 궁금해서요. 혹시 구매하신 지 얼마나 되셨나요?
================================ Human Message =================================

3년 됐어..
================================== Ai Message ==================================
Tool Calls:
  record_warranty_status (call_KsNimA7FlFDFsKwfDiC74OC2)
 Call ID: call_KsNimA7FlFDFsKwfDiC74OC2
  Args:
    status: out_of_warranty
================================= Tool Message =================================
Name: record_warranty_status

보증 상태가 확인되었습니다: out_of_warranty
=======